# Programming in julia/Hecke/Oscar - Session 1

## Julia control flow

Julia is an imparative language with all the known workflow constructions.

### If conditions

In [1]:
function f(x)
    if x > 0
        return false
    elseif x < 0
        return true
    else
        error("input must be nonzero")
    end
end

f(2)

false

In [2]:
f(0)

LoadError: input must be nonzero

### For loops

In [2]:
function f(x)
    for i in 1:10
        if i == 2
            continue
        end
        x += i # this is the same as x = x + i
    end
    return x
end

f(0.5)

53.5

### Range objects

Note that `1:10` is a range object, which represents the list `1,2,...,10`. Ranges come in different flavors. If one really needs the array of the elements in the range, one can use `collect`.

In [4]:
x = 1:10
println(typeof(x))
println(collect(x))
println(collect(1:2:10))
println(collect(10:-3:2))

UnitRange{Int64}
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[1, 3, 5, 7, 9]
[10, 7, 4]


### While loops

In [1]:
function f(x)
    while x < 100
        x = x^2
        if x < 50
            break
        end
    end
    return x
end

f(5)

25

### Ternary conditional operator

In [6]:
function f(x)
    return isodd(x) ? -1 : 1
end

f(3), f(2)

(-1, 1)

## Basic julia types

### `Int` (aka `Int64`)
Values of this type are usually called machine integers, which is roughly equivalent to working modulo $2^{64}$.

In [7]:
x = 1
typeof(x)

Int64

In [8]:
x = 2^64

0

In [9]:
Int === Int64

true

### `Float64`

These are classical floating point numbers (with 53 bits of precision), which are also known as `double`.

In [10]:
x = 1.0
typeof(x)

Float64

### `String`

In [11]:
x = "a + b"
typeof(x)

String

There is something called string interpolation, which is quite useful when quickly printing some things:

In [12]:
x = 3 # this is an Int
println("x is of type $(typeof(x)) with value $(x)")

x is of type Int64 with value 3


### `BigInt`

These is integers with arbitrary size.

In [13]:
x = BigInt(2)^100

1267650600228229401496703205376

### `Vector`
This is the basic types for lists. Note that the fullname is `Vector{T}`, where `T` is the type of the elements of the list. Here are some useful ways to construct lists.

In [14]:
x = [1, 2, 3]

3-element Array{Int64,1}:
 1
 2
 3

In [15]:
v = Vector{Int}(undef, 3) # Create a list for values of type `Int` with length 3. The entries in `v` are undefined.
v[1] = 1; v[2] = -1; v[3] = 4
v

3-element Array{Int64,1}:
  1
 -1
  4

In [16]:
v = zeros(Int, 4)

4-element Array{Int64,1}:
 0
 0
 0
 0

### `Matrix`
This is the basic type for 2-dimensional arrays. As for `Vector`, it actually is `Matrix{T}`, where `T` is the type of the elements of the list. Here are some constructors:

In [17]:
x = [1 2; 3 4]

2×2 Array{Int64,2}:
 1  2
 3  4

In [1]:
v = Matrix{Int}(undef, 2, 3)
v[1, 1] = 1; v[1, 2] = 2; v[1, 3] = 3; v[2, 1] = 4; v[2, 2] = 5; v[2, 3] = 6;
v

2×3 Array{Int64,2}:
 1  2  3
 4  5  6

### `Array`
These are multidimensional arrays, have full type `Array{T, N}`, where `T` is the type of the elements and `N` is the dimension. In fact, by definition we have `Vector{T} === Array{T, 1}` and `Matrix{T} === Array{T, 2}`.

Of course, all of these constructions can be nested:

In [19]:
x = [[1, 2], [3, 4]]

2-element Array{Array{Int64,1},1}:
 [1, 2]
 [3, 4]

## Methods and multiple dispatch

One of the must useful features of julia is the ability to define multiple methods for the same function combined with multiple dispatch. It usually takes some time appreciate it. This also means that one should rarely do stuff like `if typeof(x) == ...`.

In [20]:
function f(x)
    y = x^2
    if x isa Int
        return y - 1
    else
        return y + 1
    end
end

f(2), f(2.0)

(3, 5.0)

This is <b>bad</b>. Better:

In [21]:
function g(y::Int)
    return y - 1
end

function g(y) # this is the same as g(y::Any)
    return y + 1
end

function f(x)
    y = x^2
    return g(y)
end

f(2), f(2.0)

(3, 5.0)

This works, because julia will always use the more specific method (the <i>function</i> `g` has two <i>methods</i>). Here is another example.

In [22]:
Int <: Integer <: Any

true

The type `Int` is a subtype of `Integer`, which is a subtype of `Any`.

In [23]:
function f(x) # same as f(x::Any)
    return x + 1
end

function f(x::Integer)
    return x + 2
end

function f(x::Int)
    return x + 3
end

f(2.0), f(Int32(2)), f(2) # typeof(3) == Int and Int32 <: Integer

(3.0, 4, 5)

Of course this can also be used to allow only specific types

In [34]:
function f(x::Int)
    return x^2
end

f(2)

4

In [35]:
f(1.0)

LoadError: MethodError: no method matching f(::Float64)
Closest candidates are:
  f(!Matched::Int64) at In[34]:1

This gives a `MethodError`, which tells us that julia knows of no method of `f` with accepts a `Float64`. To help us, julia will print methods of `f` that are known and why they don't match with our required arguments.

#### Introspection

Having a function with multiple methods is quite powerful, but sometimes makes it hard to figure out which method is actually called. Luckily julia has some very useful tool for type introspection, namely the macro `@which` (we won't discuss what a macro actually is for now).

In [39]:
function f(x::Number)
   return x^2
end

function f(x::Integer)
   return x + 1
end

f (generic function with 3 methods)

In [40]:
@which f(2.0)

f(x::Number) in Main at In[39]:1

In [42]:
@which f(1)

f(x::Int64) in Main at In[36]:1

This works for almost any function:

In [43]:
@which 1 + 1

+(x::T, y::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} in Base at int.jl:86

In [44]:
@which push!([1, 2], 2)

push!(a::Array{T,1}, item) where T in Base at array.jl:932

A similar macro is `@less`, which actually shows the source code of the method that will be called.

## Oscar/Hecke types

The julia Basic types are an indespensible tool when programming in julia. But of course they are not sufficient for doing computer algebra, which requires univariate and multivaraite polynomials, power series, residue rings, residue fields or matrices.

### Philosophy

Similar to Sage and Magma, in the Oscar ecosystem everything has parents. In other words, elements of a ring/group/field do not exist alone, but every element has a parent. In fact very often, one first has to construct the parent before one can construct elements. Here is an example to construct the polynomial $x^2 + 2x \in \mathbf{Z}[x]$:

In [2]:
using Hecke;
Zx, x = ZZ["x"] # returns the polynomial ring together with the variable
h = x^2 + 2x

x^2 + 2*x

### Basic types
Here are some common basic rings that one would like to construct.

#### The ring of integers: $\mathbf Z$

In [13]:
R = ZZ

Integer Ring

#### The field of rationals: $\mathbf Q$

In [14]:
F = QQ

Rational Field

#### Finite prime fields:  $\mathbf F_p$ 

In [15]:
R = GF(3)

Galois field with characteristic 3

#### Finite fields: $\mathbf F_{p^r}$

In [16]:
F, a = FiniteField(3, 2, "a")

(Finite field of degree 2 over F_3, a)

Note that the second return value is an algebra generator for $\mathbf F_{p^r}$ over $\mathbf F_p$, that is, $\mathbf F_{p^r} = \mathbf F_p[a]$.

#### Residue rings of integers: $\mathbf Z/n\mathbf Z$

In [17]:
R = ResidueRing(ZZ, 4)

Integers modulo 4

#### Number fields: $\mathbf Q(\alpha)$

In [12]:
Qx, x = QQ["x"] # alternative syntax for PolynomialRing(QQ, "x")
h = x^2 + 1
K, a = NumberField(h, "a")

(Number field over Rational Field with defining polynomial x^2 + 1, a)

The second return value is the class of $x$ in $\mathbf Q[x]/(h)$.

### Derived rings
A second class of rings is obtained by taking old rings to create new rings or parents.

#### Univariate polynomial rings: $R[x]$

In [18]:
R = ZZ
Rx, x = ZZ["x"]

(Univariate Polynomial Ring in x over Integer Ring, x)

#### Multivariate polynomial rings: $R[x_1,\dotsc,x_n]$

In [23]:
R = ZZ
Rx, (x1, x2) = ZZ["x1", "x2"]

(Multivariate Polynomial Ring in x1, x2 over Integer Ring, fmpz_mpoly[x1, x2])

#### Power series ring: $R[[x]]$

In [21]:
R = ZZ
Rx, x = PowerSeriesRing(R, 30, "x")

(Univariate power series ring in x over Integer Ring, x + O(x^31))

#### Matrix spaces: $\operatorname{M}_{n\times m}(R)$

In [22]:
R = ZZ
S = MatrixSpace(ZZ, 2, 3)

Matrix Space of 2 rows and 3 columns over Integer Ring

### Remark on matrices

1. As we saw, elements are usually constructured. For matrices this is a bit cumbersome. Thus there is a way to directly construct a matrix over a given ring:

In [30]:
z = matrix(ZZ, [1 2 3; 4 5 6]) # give a two-dimensional julia array

[1  2  3]
[4  5  6]

In [31]:
z = matrix(ZZ, 2, 3, [1, 2, 3, 4, 5, 6]) # give a one-dimensional julia array and read it row-wise

[1  2  3]
[4  5  6]

2. Matrices? As we saw, we already have some matrix type provided by julia, known as `Matrix`. Unfortunately, due to julias numerical linear algebra heritage, using the `Matrix` for linear algebra is only useful if one does linear algebra with floating point numbers. There are some dubious design decisions which render the `Matrix` useless for linear algebra.

In [25]:
using LinearAlgebra
x = [1 2; 3 4]
det(x)

-2.0

Note that this is a `Float64`. Julia promotes the matrix to matrix of `Float64` and then does the computations.   There are additional internal design decisions, which make it even less useful.

On other hand:

In [25]:
x = matrix(ZZ, [1 2; 3 4]) # this is a Oscar matrix
det(x)

-2

> Don't use the type `Matrix` to do linear algebra (unless you are doing numerical linear algebra). Use `Matrix` instead only for bookkeeping. To do exact linear algebra, use the matrix types provided by `Oscar/Hecke`.


### What can we do with elements?

Please have a look at: https://nemocas.github.io/AbstractAlgebra.jl/latest/

For example, https://nemocas.github.io/AbstractAlgebra.jl/latest/polynomial/ shows all the methods that (should) work with univariate polynomial rings.

A very concise overview of the functionality for univariate polynomials can be found here:
http://www.thofma.com/software/univariate.html

## Boosting productivity using Revise

Assume that one is working on a file `test.jl`. One usually has a julia session open and then iterates the following steps:

1. Make changes to `test.jl`.
2. run `include("test.jl")` in the julia session.
3. Test if the changes lead to the desired outcome.

This works quite well, but of course it does not scale. What if I have more than one file? Also it does not work when working directly on Hecke/Oscar?

This is where the package "Revise" comes into play, which tracks local file changes and automatically loads functions/files that have changed. This basically eliminates step 2. First step is to install Revise using the package manager by invoking `]add Revise`. Once this is done, we load Revise and tell it to track our file "test.jl"

```julia
using Revise
Revise.includet("test.jl")
```

Julia development without Revise is very cumbersome! Always use Revise!

## Some gotchas and tipps:
- Typing `1/2` will produce `0.5`, which is a floating point number. The correct way is to use `//`, so `1//2` or `x//(x + 1)` rational numbers or a rational function.
- Type `?bla` to get some information about the function `bla`.
- Use tab completion:
  - `gcd\tab` will print all functions whose name start with `gcd`. (This also works with variables in a session.)
  - `f(a, \tab` will print the methods that work with `a` as the first argument for `f` .
  - Sometimes methods are not exported, so one has to prefix them, like `Hecke.isprime_power`.
- Use `@show f` to do some quick debugging.